##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TFX Pipeline 和 TensorFlow Transform 进行特征工程

***转换输入数据并使用 TFX 流水线训练模型。***

注：我们建议在 Colab 笔记本中运行本教程，无需进行设置！只需点击“在 Google Colab 中运行”

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_tft"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_transform.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_transform.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/penguin_transform.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

在这个基于笔记本的教程中，我们将创建并运行 TFX 流水线来采集原始输入数据并对其进行适当的预处理以进行 ML 训练。此笔记本基于我们在[使用 TFX Pipeline 和 TensorFlow Data Validation 进行数据验证教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_tfdv)中构建的 TFX 流水线。如果您尚未阅读该教程，应在继续使用此笔记本之前阅读。

您可以通过特征工程化提高数据的预测质量和/或降低维度。使用 TFX 的好处之一是您只需编写一次转换代码，生成的转换将在训练和应用之间保持一致，以避免训练/应用偏差。

我们将向流水线中添加一个 `Transform` 组件。Transform 组件是使用 [tf.transform](https://tensorflow.google.cn/tfx/transform/get_started) 库实现的。

要了解有关 TFX 中各种概念的更多信息，请参阅[了解 TFX 流水线](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)。

## 安装

我们首先需要安装 TFX Python 软件包并下载将用于模型的数据集。

### 升级 Pip

为了避免在本地运行时升级系统中的 Pip，请检查以确保在 Colab 中运行。当然，可以对本地系统单独升级。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 安装 TFX


In [ ]:
!pip install -U tfx

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

### 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上面的代码单元时必须重新启动运行时，方法是单击上面的“重新启动运行时”按钮或使用“运行时 &gt; 重新启动运行时...”菜单。这样做的原因是 Colab 加载软件包的方式。

检查 TensorFlow 和 TFX 版本。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 设置变量

有一些变量用于定义流水线。您可以根据需要自定义这些变量。默认情况下，流水线的所有输出都将在当前目录下生成。

In [ ]:
import os

PIPELINE_NAME = "penguin-transform"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 准备示例数据

我们将下载要在我们的 TFX 流水线中使用的示例数据集。我们要使用的数据集为 [Palmer Penguins 数据集](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)。

然而，与之前使用已经预处理的数据集的教程不同，我们将使用**原始** Palmer Penguins 数据集。


因为 TFX ExampleGen 组件从目录中读取输入，所以我们需要创建一个目录并将数据集复制到其中。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_path = 'https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_path, _data_filepath)

快速浏览一下原始数据的样子。

In [ ]:
!head {_data_filepath}

有一些缺失值的条目表示为 `NA`。我们将在本教程中删除这些条目。

In [ ]:
!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

您应该能够看到描述企鹅的七个特征。我们将使用与之前教程相同的一组特征 -“culmen_length_mm”、“culmen_depth_mm”、“flipper_length_mm”、“body_mass_g”，并将预测企鹅的“物种”。

**唯一的区别是输入数据没有经过预处理。**请注意，我们不会在本教程中使用其他特征，例如“island”或“sex”。

### 准备架构文件

如[使用 TFX Pipeline 和 TensorFlow Data Validation 进行数据验证教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_tfdv)中所述，我们需要数据集的架构文件。因为数据集与上一个教程不同，所以我们需要再次生成它。在本教程中，我们将跳过这些步骤，只使用准备好的架构文件。


In [ ]:
import shutil

SCHEMA_PATH = 'schema'

_schema_uri = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/schema/raw/schema.pbtxt'
_schema_filename = 'schema.pbtxt'
_schema_filepath = os.path.join(SCHEMA_PATH, _schema_filename)

os.makedirs(SCHEMA_PATH, exist_ok=True)
urllib.request.urlretrieve(_schema_uri, _schema_filepath)

此架构文件是使用与上一教程中相同的流水线创建的，无需任何手动更改。

## 创建流水线

TFX 流水线是使用 Python API 定义的。我们将向已在 [Data Validation 教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_tfdv)中创建的流水线中添加 `Transform` 组件。

Transform 组件需要来自 `ExampleGen` 组件的输入数据和来自 `SchemaGen` 组件的架构，并生成“转换图”。输出将用于 `Trainer` 组件。另外，Transform 还可以选择性地生成“转换后的数据”，也就是转换后的具体化数据。但是，我们将在本教程中的训练期间转换数据，而不会具体化中间转换数据。

需要注意的一点是，我们需要定义一个 Python 函数 (`preprocessing_fn`) 来描述应如何转换输入数据。这类似于 Trainer 组件，同样需要用户代码来定义模型。


### 编写预处理和训练代码

我们需要定义两个 Python 函数。一个用于 Transform，一个用于 Trainer。

#### preprocessing_fn

Transform 组件将在给定的模块文件中找到一个名为 `preprocessing_fn` 的函数，就像我们针对 `Trainer` 组件所做的那样。您还可以使用 Transform 组件的 <a data-md-type="raw_html" href="https://github.com/tensorflow/tfx/blob/142de6e887f26f4101ded7925f60d7d4fe9d42ed/tfx/components/transform/component.py#L113">`preprocessing_fn` 参数</a>指定具体函数。

在本例中，我们将进行两种转换。对于像 `culmen_length_mm` 和 `body_mass_g` 这样的连续数字特征，我们将使用 [tft.scale_to_z_score](https://tensorflow.google.cn/tfx/transform/api_docs/python/tft/scale_to_z_score) 函数对这些值进行规范化。对于标签特征，我们需要将字符串标签转换为数字索引值。我们将使用 [`tf.lookup.StaticHashTable`](https://tensorflow.google.cn/api_docs/python/tf/lookup/StaticHashTable) 进行转换。

为了轻松找出转换后的字段，我们将 `_xf` 后缀附加到转换后的特征名称。

#### run_fn

模型本身与之前的教程几乎相同，但这次我们将使用来自 Transform 组件的转换图来转换输入数据。

与上一教程相比，一个更重要的区别是，我们现在导出了用于提供服务的模型，其不仅包括模型的计算图，还包括在 Transform 组件中生成的用于预处理的转换图。我们需要定义一个单独的函数来为传入的请求提供服务。您可以看到，训练数据和为请求提供服务使用的是同一函数 `_apply_preprocessing`。


In [ ]:
_module_file = 'penguin_utils.py'

In [ ]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

# Specify features that we will use.
_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 3 possible values, we use a hard-coded
  # table for simplicity.
  table_keys = ['Adelie', 'Chinstrap', 'Gentoo']
  initializer = tf.lookup.KeyValueTensorInitializer(
      keys=table_keys,
      values=tf.cast(tf.range(len(table_keys)), tf.int64),
      key_dtype=tf.string,
      value_dtype=tf.int64)
  table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://tensorflow.google.cn/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

现在，您已完成构建 TFX 流水线的所有准备步骤。

### 编写流水线定义

我们定义一个函数来创建 TFX 流水线。`Pipeline` 对象表示 TFX 流水线，可使用 TFX 支持的流水线编排系统之一来运行。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## 运行流水线

我们将像上一教程一样使用 `LocalDagRunner`。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

如果流水线成功完成，您应该看到“INFO:absl:Component Pusher is finished.”。

如果您在之前的步骤中未更改变量，Pusher 组件则会将经过训练的模型推送到 `SERVING_MODEL_DIR`，即 `serving_model/penguin-transform` 目录。您可以从 Colab 左侧面板中的文件浏览器查看结果，或者使用以下命令：

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

您还可以使用 [`saved_model_cli` 工具](https://tensorflow.google.cn/guide/saved_model#show_command)检查生成的模型的签名。

In [ ]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default

因为我们使用自己的 `serve_tf_examples_fn` 函数定义了 `serving_default`，所以签名显示它采用单个字符串。此字符串是 tf.Examples 的序列化字符串，将使用我们之前定义的 [tf.io.parse_example()](https://tensorflow.google.cn/api_docs/python/tf/io/parse_example) 函数进行解析（在[此处](https://tensorflow.google.cn/tutorials/load_data/tfrecord)了解有关 tf.Examples 的更多信息）。

我们可以加载导出的模型，并通过几个示例尝试一些推断。

In [ ]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path

loaded_model = tf.keras.models.load_model(model_path)
inference_fn = loaded_model.signatures['serving_default']

In [ ]:
# Prepare an example and run inference.
features = {
  'culmen_length_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
  'culmen_depth_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[16.1])),
  'flipper_length_mm': tf.train.Feature(int64_list=tf.train.Int64List(value=[213])),
  'body_mass_g': tf.train.Feature(int64_list=tf.train.Int64List(value=[5400])),
}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

对应于“Gentoo”物种的第三个元素预计是三个元素中最大的。

## 后续步骤

如果您想了解有关 Transform 组件的更多信息，请参阅 [Transform 组件指南](https://tensorflow.google.cn/tfx/guide/transform)。您可以在 https://tensorflow.google.cn/tfx/tutorials 上找到更多资源。

要了解有关 TFX 中各种概念的更多信息，请参阅[了解 TFX Pipelines](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)。
